In [1]:
import time
from multiverse import MeasurementPlaneClient

def on_result_callback(result):
    print(f"New Result Received: {result}")

def on_completion_callback(status):
    print(f"Measurement Completed wit   h Status: {status}")

BROKER_URL = "http://localhost:5672/"
# Step 1: Get the capabilities published by agents
mpClient = MeasurementPlaneClient(BROKER_URL)

Give it some time to start loading the available capabilities

In [8]:
capabilities = mpClient.get_capabilities(capability_type="measure")
print("Available Capabilities:", capabilities)

Available Capabilities: {0: {'label': 'region time Capability', 'endpoint': '/region_time_endpoint', 'capabilityName': 'region_time_capability', 'parameters_schema': {'type': 'object', 'properties': {'region': {'type': 'string', 'description': 'The region of requested time'}}}, 'resultSchema': {'type': 'object', 'properties': {'region': {'type': 'string', 'description': 'The region of requested time'}, 'local time': {'type': 'string', 'description': 'The local time'}}}, 'timestamp': '2024-10-16 17:01:49.27', 'nonce': '12345', 'metadata': None, 'capability': 'measure'}}


In [13]:
capability_id = 0
capability = capabilities[capability_id]
# Step 3: Instantiate a specification of a measurement from the capability
measurement = mpClient.create_measurement(capability)
print(f"Created Measurement: {measurement.specification_message}")

# Step 4: Configure the measurement with parameters
schedule = "now | 2024-11-15 22:50:00 |stream"  # Replace with your schedule: `now / period`, `start ... stop / period`, etc.
parameters = {
    "region": 'Europe/London'
    # Add more parameters based on the capability.parameters_schema JSON schema
}
measurement.configure(
    schedule=schedule,
    parameters=parameters,
    stream_results=False,  # Stream results
    redirect_to_storage=True,  # Redirect to storage
    result_callback=on_result_callback,  # Callback function for new results
    completion_callback=on_completion_callback  # Callback function for measurement completion
)
print(f"Configured Measurement Specification: {measurement.specification_message}")

# Step 5: Send the Measurement
mpClient.send_measurement(measurement)




Created Measurement: {'label': 'region time Capability', 'endpoint': '/region_time_endpoint', 'capabilityName': 'region_time_capability', 'parameters_schema': {'type': 'object', 'properties': {'region': {'type': 'string', 'description': 'The region of requested time'}}}, 'resultSchema': {'type': 'object', 'properties': {'region': {'type': 'string', 'description': 'The region of requested time'}, 'local time': {'type': 'string', 'description': 'The local time'}}}, 'timestamp': '2024-10-16 17:01:49.27', 'nonce': '12345', 'metadata': None, 'specification': 'measure'}
Configured Measurement Specification: {'label': 'region time Capability', 'endpoint': '/region_time_endpoint', 'capabilityName': 'region_time_capability', 'parameters_schema': {'type': 'object', 'properties': {'region': {'type': 'string', 'description': 'The region of requested time'}}}, 'resultSchema': {'type': 'object', 'properties': {'region': {'type': 'string', 'description': 'The region of requested time'}, 'local time':

New Result Received: [{'region': 'Europe/London', 'local time': '2024-10-16 22:13:01 BST+0100'}]
New Result Received: [{'region': 'Europe/London', 'local time': '2024-10-16 22:13:01 BST+0100'}]
New Result Received: [{'region': 'Europe/London', 'local time': '2024-10-16 22:13:02 BST+0100'}]
New Result Received: [{'region': 'Europe/London', 'local time': '2024-10-16 22:13:02 BST+0100'}]
New Result Received: [{'region': 'Europe/London', 'local time': '2024-10-16 22:13:03 BST+0100'}]
New Result Received: [{'region': 'Europe/London', 'local time': '2024-10-16 22:13:03 BST+0100'}]


In [14]:

# Step 6: Interrupt the measurement
mpClient.interrupt_measurement(measurement)
print(f"Measurement Interrupted")

Measurement Interrupted
